In [ ]:
import re
import sys

import numpy as np

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagate into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
import pymedphys._utilities.transforms

In [ ]:
if sys.platform == 'win32':
    import os
    import win32file
    import msvcrt
    
    def read_trf_contents(filepath):
        handle = win32file.CreateFile(
            str(filepath),
            win32file.GENERIC_READ,
            win32file.FILE_SHARE_DELETE |
            win32file.FILE_SHARE_READ |
            win32file.FILE_SHARE_WRITE,
            None,
            win32file.OPEN_EXISTING,
            0,
            None)


        detached_handle = handle.Detach()

        file_descriptor = msvcrt.open_osfhandle(
            detached_handle, os.O_RDONLY)

        with open(file_descriptor) as file:
            data = file.read()
            
        return data
    
else:
    def read_trf_contents(filepath):
        with open(filepath, 'r') as file:
            data = file.read()
            
        return data

In [ ]:
data_paths = pymedphys.zip_data_paths("monaco_trf_compare.zip")
data_paths

In [ ]:
vmat_with_col_paths = [path for path in data_paths if path.parent.name == "StaticAnglesNoCol"]
vmat_with_col_paths

In [ ]:
def get_file_type(input_paths, file_type):
    paths = [path for path in input_paths if file_type in path.name]
    assert len(paths) == 1
    return paths[0]
    

tel_path = get_file_type(vmat_with_col_paths, "tel")
dcm_path = get_file_type(vmat_with_col_paths, "dcm")
# dcm_path

In [ ]:
delivery_dcm = pymedphys.Delivery.from_dicom(
    pydicom.read_file(str(dcm_path), force=True))

In [ ]:
data = read_trf_contents(tel_path)

In [ ]:
mlc_pos_pattern = ' *-?\d+\.\d'
ten_mlc_pos_pattern = ",".join([mlc_pos_pattern,]*10)
sixteen_rows_of_mlcs_pattern = "\n".join([ten_mlc_pos_pattern,]*16)

In [ ]:
weird_ones = "\n".join([",".join(["1",]*6),]*13)

decimal_param = "-?\d+\.\d+"
optional_decimal_param = "-?\d+(?:\.\d+)?"

parameters = (
    "1,1\n"
    f"{decimal_param},({optional_decimal_param})\n"
    f"0\n{decimal_param},{decimal_param},{decimal_param},{decimal_param}\n"
    f"({decimal_param}),{decimal_param},{decimal_param},{decimal_param}\n"
    f"{optional_decimal_param},({optional_decimal_param}),{optional_decimal_param},({optional_decimal_param})"
)

weird_zeros = "       0\n0\n0"

In [ ]:
# total_pattern = f"({sixteen_rows_of_mlcs_pattern})\n{weird_ones}\n({parameters})\n{weird_zeros}"
total_pattern = f"({sixteen_rows_of_mlcs_pattern})\n{weird_ones}\n{parameters}"
all_controlpoint_results = re.findall(total_pattern, data)

In [ ]:
# all_controlpoint_results

In [ ]:
delivery_dcm.mu

In [ ]:
mu = np.cumsum([float(result[2]) for result in all_controlpoint_results])
mu

In [ ]:
np.allclose(mu, delivery_dcm.mu)

In [ ]:
iec_gantry_angle = [float(result[1]) for result in all_controlpoint_results]
iec_gantry_angle

In [ ]:
bipolar_gantry_angle = pymedphys._utilities.transforms.convert_IEC_angle_to_bipolar(iec_gantry_angle)
bipolar_gantry_angle

In [ ]:
np.allclose(bipolar_gantry_angle, delivery_dcm.gantry)

In [ ]:
# jaws = [(float(result[3]), float(result[4])) for result in all_controlpoint_results]
# jaws

In [ ]:
jaw_gap = np.array([float(result[3]) for result in all_controlpoint_results])
jaw_field_centre = np.array([float(result[4]) for result in all_controlpoint_results])

In [ ]:
jaw_a = jaw_field_centre + jaw_gap/2
jaw_a

In [ ]:
jaw_b = -(jaw_field_centre - jaw_gap/2)
jaw_b

In [ ]:
jaws = np.vstack([jaw_a, jaw_b]).T
jaws

In [ ]:
np.allclose(jaws, delivery_dcm.jaw)

In [ ]:
mlc_strings = [result[0] for result in all_controlpoint_results]
an_mlc_string = mlc_strings[0]

In [ ]:
def convert_mlc_string(mlc_string):
    mlcs = np.array(mlc_string.replace(' ', '').replace('\n',',').split(',')).astype(float)
    mlcs = mlcs.reshape((80,2))

    mlcs[:,0] = -mlcs[:,0]
    mlcs = np.fliplr(np.flipud(mlcs))
    
    return mlcs


mlcs = [convert_mlc_string(result[0]) for result in all_controlpoint_results]

In [ ]:
np.allclose(mlcs, delivery_dcm.mlc)